This notebook is used to enable search across all the notebooks if you don't want to use grep or something simliar

<b>How to set this up</b>

Details tbc

In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch, RequestsHttpConnection
#es = Elasticsearch()

es = Elasticsearch(hosts=[{"host": "host.docker.internal", "port": 9200}], connection_class=RequestsHttpConnection, max_retries=30,
                       retry_on_timeout=True, request_timeout=30)
doc = {
    'author': 'JONES',
    'text': 'Elasticsearch: cool. bonsai cool. blah',
    'timestamp': datetime.now(),
}


res = es.index(index="test-index", id=2, body=doc)
print(res['result'])

res = es.get(index="test-index", id=1)
print(res['_source'])

es.indices.refresh(index="test-index")


# print("Got %d Hits:"aa % res['hits']['total']['value'])
# for hit in res['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

ConnectionError: ConnectionError(HTTPConnectionPool(host='host.docker.internal', port=9200): Max retries exceeded with url: /test-index/_doc/2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3fa0d05be0>: Failed to establish a new connection: [Errno 111] Connection refused'))) caused by: ConnectionError(HTTPConnectionPool(host='host.docker.internal', port=9200): Max retries exceeded with url: /test-index/_doc/2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3fa0d05be0>: Failed to establish a new connection: [Errno 111] Connection refused')))

In [ ]:
connect_elasticsearch()

In [ ]:
import json
r = requests.get('host.docker.internal:9200') 
# i = 1
# while r.status_code == 200:
#     r = requests.get('http://swapi.co/api/people/'+ str(i))
#     es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
#     i=i+1
 
# print(i)

In [ ]:
res = es.get(index="test-index", id=1)
print(res['_source'])